## PLIP — Protein ligand interaction profiling
This notebook will walk us through using PLIP for identification of non-covalent interactions between biological macrmolecules and their ligands.

PLIP implementation paper:
Adasme et al. PLIP 2021: expanding the scope of the protein-ligand interaction profiler to DNA and RNA. NAR 2021 

## 0) Setup
See [Quickstart](../index.ipynb#imports) for more details on the setup.


## 0.0) Imports

In [17]:
import os
import json
from pathlib import Path

import requests
from pdbtools import pdb_fetch
import tengu

## 0.1) Credentials

In [18]:
TOKEN = os.getenv("TENGU_TOKEN")
# You might have a custom deployment url, by default it will use https://tengu.qdx.ai
TENGU_URL = os.getenv("TENGU_URL") or "https://tengu.qdx.ai"
print(TENGU_URL)

https://nginx-reverse-proxy-staging-edh4uref5a-as.a.run.app


## 0.2) Configuration
Let's set some global variables that define our project

In [19]:
DESCRIPTION = "plip-inference-notebook"
TAGS = ["tengu-py", "plip", "notebook"]

WORK_DIR = Path.home() /  "qdx" / "plip-inference-notebook"

## 0.3) Build your tengu client

In [20]:
#|hide
if WORK_DIR.exists():
    client = tengu.Provider(workspace=WORK_DIR, access_token=TOKEN, url=TENGU_URL)
    await client.nuke()

In [21]:
os.makedirs(WORK_DIR, exist_ok=True)

client = await tengu.build_provider_with_functions(
    access_token=TOKEN, url=TENGU_URL, workspace=WORK_DIR, batch_tags=TAGS
)

## 0.4 Fetch example PDB

In [22]:
PDB_ID = "1B39"
FILE_NAME = f'{PDB_ID}.pdb'
FILE_PATH = WORK_DIR / FILE_NAME
complex = list(pdb_fetch.fetch_structure(PDB_ID))

with open(FILE_PATH, 'w') as f:
    for line in complex:
        f.write(line)

## 0.5) Convert PDB file to QDXF format
QDXF is the central molecule format of the Tengu API, so before we use the Dubai module to infer connectivity (bonds) for our molecule, we must convert our PDB file to QDXF.

In [27]:
# # We need to specify storage > file size to ensure that we have allocated enough resources for the convert module

FILE_SIZE = os.path.getsize(FILE_PATH) * 2
(protein,) = await client.convert("PDB", FILE_PATH, target="NIX", resources={"storage": FILE_SIZE})

protein = await protein.get()

# 1.0) Check Tengu API docs to get more information on the PLIP module
Before we call the PLIP Tengu module, we can look at the help documentation to get more information on the required inputs, expected outputs, and possible configuration parameters.


In [28]:
await client.plip(protein, resources={"storage": FILE_SIZE})

AttributeError: 'Provider' object has no attribute 'plip'